In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya
model = malaya.segmentation.transformer(model = 'small')

In [3]:
import json

with open('prepare/test-set-segmentation.json') as fopen:
    data = json.load(fopen)
    
X, Y = [], []
for x, y in data:
    X.append(x)
    Y.append(y)

In [4]:
model.greedy_decoder(X[:10])

['Nihon bashi Mitsui Tower. Menara Nihon bashi Mitsui (Ri Ben Qiao Sanumber tawa, "Nihon bashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo,umber Jepun. Dengan ketinggian 192 meter, bangunan berkembang berkembangnya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen M menghukum shimae, Tokyo berkembang berkembang .',
 'nihon bashi mitsui tower. menara nihon bashi mitsui (ri ben qiao san jing tawa, "nihon bashi mtusi tawa") merupakan sebuah bangunan pencakar langit yang terletak di chuo, tokyo, jepun. dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. berkembang telah siap pada tahun 2005. ia dihubungkan oleh stesen bawah tanah, stesen mitsukoshimae, tokyo metro.',
 'Nihon bashi Mitsui Tower. Menara Nihon bashi Mitsui(Ri Ben Qiao San Jing tawa," Nihon bashi MtusiTawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo, Tokyo, Jepun. Dem be

In [5]:
from tqdm import tqdm

batch_size = 10
results = []
for i in tqdm(range(0, len(X), batch_size)):
    batch_x = X[i: i + batch_size]
    r = model.greedy_decoder(batch_x)
    results.extend(r)

100%|██████████| 9939/9939 [44:21<00:00,  3.73it/s]   


In [6]:
results_Y = results[:]
results_Y[0], Y[0]

('Nihon bashi Mitsui Tower. Menara Nihon bashi Mitsui (Ri Ben Qiao Sanumber tawa, "Nihon bashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo,umber Jepun. Dengan ketinggian 192 meter, bangunan berkembang berkembangnya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen M menghukum shimae, Tokyo berkembang berkembang .',
 'Nihonbashi Mitsui Tower. Menara Nihonbashi Mitsui (Ri Ben Qiao San Jing tawa, "Nihonbashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo, Tokyo, Jepun. Dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. Pembinaannya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen Mitsukoshimae, Tokyo Metro.')

In [7]:
def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [10]:
wer = []
for i in tqdm(range(len(results_Y))):
    wer.append(calculate_wer(Y[i], results_Y[i]))

100%|██████████| 99384/99384 [00:03<00:00, 28508.08it/s]


In [11]:
import numpy as np
np.mean(wer)

0.20852097556764446